In [2]:
#install.packages("remotes")
#remotes::install_github("theislab/zellkonverter")
# Seurat installed with conda
#install.packages('rdist') # DO this weird armadillo bugfix for MacOS
#install.packages('energy')

In [63]:
# !wget https://zenodo.org/record/7041849/files/DatlingerBock2021.h5ad
sce_object <- zellkonverter::readH5AD('DatlingerBock2021.h5ad', X_name='counts')
seurat_object <- Seurat::as.Seurat(sce_object, couunts = 'counts', data = 'counts')

Warning message:
“The following arguments are not used: couunts”


In [64]:
# # basic qc and pp
# sc.pp.filter_cells(adata, min_counts=1000)
# sc.pp.normalize_per_cell(adata)
# sc.pp.filter_genes(adata, min_cells=50)
# sc.pp.log1p(adata)

# # high class imbalance
# adata = equal_subsampling(adata, 'perturbation', N_min=50)
# sc.pp.filter_genes(adata, min_cells=3)  # sanity cleaning

# # select HVGs
# n_var_max = 2000  # max total features to select
# sc.pp.highly_variable_genes(adata, n_top_genes=n_var_max, subset=False, flavor='seurat_v3', layer='counts')
# sc.pp.pca(adata, use_highly_variable=True)
# sc.pp.neighbors(adata)

seurat_object[["percent.mt"]] <- PercentageFeatureSet(seurat_object, pattern = "^MT-")
seurat_object <- NormalizeData(seurat_object, normalization.method = "LogNormalize", scale.factor = 10000)
seurat_object <- FindVariableFeatures(seurat_object, selection.method = "vst", nfeatures = 2000)
seurat_object <- ScaleData(seurat_object, features = rownames(seurat_object))
seurat_object <- RunPCA(seurat_object, features = VariableFeatures(object = seurat_object), verbose = FALSE)

Centering and scaling data matrix



In [70]:
labels <- seurat_object[[]][['perturbation']]
pca <- Embeddings(seurat_object, reduction = "pca")
dim(pca)

[1] 39194    50

In [72]:
groups <- unique(labels)

In [74]:
x <- as.matrix(pca)[labels=='control',]
y <- as.matrix(pca)[labels==groups[1],]
X <- rbind(x, y)
d <- dist(X)

In [93]:
res <- energy::eqdist.etest(d, sizes=c(nrow(x), nrow(y)), distance=TRUE, R = 10)


In [92]:
df <- data.frame(row.names = groups, pval = rep(NA, length(groups)))

In [102]:
for (group in groups) {
  x <- as.matrix(pca)[labels=='control',]
  y <- as.matrix(pca)[labels==group,]
  X <- rbind(x, y)
  d <- dist(X)
  res <- energy::eqdist.etest(d, sizes=c(nrow(x), nrow(y)), distance=TRUE, R = 10)
  df[group, 'pval'] <- res$'p.value'
  df[group, 'edist'] <- res$statistic
}

In [103]:
df

,pval,edist
,<dbl>,<dbl>
ZAP70_1,0.09090909,4.022742e+01
ZAP70_2,0.09090909,8.198755e+01
LCK_1,0.09090909,3.739764e+01
LCK_2,0.09090909,6.629021e+01
LAT_1,0.09090909,6.540238e+01
LAT_2,0.09090909,6.552532e+01
DOK2_1,0.09090909,3.323810e+01
DOK2_2,0.09090909,1.835377e+01
PTPN6_1,0.09090909,3.380036e+01
